In [ ]:
"""
Data Extract — Casa dos Ventos (PS Analytics)
Coleta de aerogeradores do ArcGIS (SIGEL/ANEEL) usando requests,
com paginação via resultOffset e salvando o raw JSON.

Saída:
- outputs/raw/aerogeradores_raw.geojson  (lista única com todos os features)
"""

import extract

# Se quiser, dá para filtrar aqui por atributos do serviço usando extra_params 
# Exemplo: extra_params = {"where": "UF='RN'"}
extra_params = {"units": "meters"}

print("[INFO] Iniciando coleta do SIGEL/ANEEL (ArcGIS)")
features = extract.fetch_all_features(extra_params=extra_params, verbose=True)
print(f"[INFO] Total coletado: {len(features)} registros")

In [ ]:
import processing
import os

csv_path = "outputs/aerogeradores.csv"

print("[INFO] Convertendo geoJSON para GeoDataFrame (EPSG:4326)")
gdf = processing.gdf_from_geojson(features)

print("[INFO] Padronizando nome das colunas com letras minusculas")
gdf = processing.lowercase_columns(gdf)

print("[INFO] Adicionando colunas de latitude e longitude")
gdf = processing.add_lat_lon(gdf)

print("[INFO] Passando todas as colunas de datas para o padrão utc")
gdf = processing.date_to_utc(gdf)

print("[INFO] Lidando com quebras de linha '\\r\\n' presentes nos dados")
gdf = processing.deal_with_line_breaks(gdf)

print("[INFO] Iniciando validações pré-programadas nos dados")
gdf = processing.validate_gdf(gdf)

print(f"[INFO] Número de linhas final: {len(gdf)}")
print("[INFO] Exportando em formato .csv")

os.makedirs("outputs", exist_ok=True)
gdf.to_csv(csv_path, index=False)

print(f"[OK] CSV salvo em: {csv_path}")